# Data Preprocessing and Exploration

-This notebook initiates the model selection process by exploring the data and preprocessing it for model training and furthur processing.The data used is 2 years sales data of an online retailer in the UK between December 2009 and January 2012.

In [40]:
#Import relevant libraries
import warnings
warnings.filterwarnings('ignore')
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gr

In [3]:
df1 = pd.read_excel("online_retail_II.xlsx",sheet_name="Year 2009-2010")
df2 = pd.read_excel("online_retail_II.xlsx",sheet_name="Year 2010-2011")
df1


In [4]:
print("Min Date:", df1["InvoiceDate"].min())
print("Max Date:", df1["InvoiceDate"].max())


In [5]:
print("Min Date:", df2["InvoiceDate"].min())
print("Max Date:", df2["InvoiceDate"].max())


In [6]:
df1["year"] = df1["InvoiceDate"].dt.year
df1["date"] = df1["InvoiceDate"].dt.date
df2["year"] = df2["InvoiceDate"].dt.year
df2["date"] = df2["InvoiceDate"].dt.date
df1


In [7]:
print("D1 MAX:", df1["InvoiceDate"].max())
print("D2 MIN:", df2["InvoiceDate"].min())

In [8]:
# Concatenate the DataFrames
merged_df = pd.concat([df1, df2], ignore_index=True)
merged_df

In [9]:
print("Merged max :", merged_df["InvoiceDate"].max())
print("Merged min :", merged_df["InvoiceDate"].min())

In [10]:
merged_df.to_csv("uk_dataset_full.csv")

In [11]:
merged_df["year"].value_counts()

In [12]:
merged_df["Quantity"].min()

In [13]:
merged_df = merged_df[merged_df["Quantity"]>0]
merged_df.shape

In [14]:
merged_daily = merged_df.groupby('date')['Quantity'].sum().reset_index()
merged_daily

In [15]:
plt.figure(figsize=(10,5))
plt.plot(merged_daily["date"],merged_daily["Quantity"]);

In [41]:
def find_date_range(df):
    min_date =df['date'].min()
    max_date = df['date'].max()
    date_range = pd.date_range(start=min_date, end=max_date)
    df = df.set_index('date').reindex(date_range).rename_axis('date').reset_index()
    return  df
    

In [16]:
df = find_date_range(merged_daily)
df.shape

In [17]:
merged_daily.shape

In [18]:
df.isna().sum()

In [19]:
df.interpolate(method='linear', inplace=True)

In [20]:
df['Quantity'] = df['Quantity'].ffill()
df.isna().sum()

In [21]:
plt.figure(figsize=(16,7))
plt.plot(merged_daily["date"],merged_daily["Quantity"])


In [22]:
plt.figure(figsize=(16,7))
plt.plot(df["date"],df["Quantity"]);

In [23]:
# df.to_csv("uk_daily_sales_full.csv")

## UK Dataset Preprocessing

In [25]:
df = pd.read_csv("uk_daily_sales_full.csv")
df.isna().sum()

In [26]:
df = pd.read_csv("uk_daily_sales_full.csv", parse_dates=['date'], index_col='date')
df =df.asfreq('D')
df.head(10)

In [27]:
def plot_graph(data):
    plt.figure(figsize=(12,6))
    plt.plot(data["quantity"]);

In [28]:
df = df[["Quantity"]]
df.rename(columns={"Quantity":"quantity"},inplace=True)
df.head(5)

In [29]:
import holidays
# Function to create date features
def create_date_features(data):
    data["month"] = data.index.month
    data["day_of_month"] = data.index.day
    data["is_month_start"] = data.index.is_month_start.astype(int)
    data["is_month_end"] = data.index.is_month_end.astype(int)
    data["day_of_year"] = data.index.dayofyear
    data["week_of_year"] = data.index.isocalendar().week
    data["day_of_week"] = data.index.dayofweek + 1
    data["year"] = data.index.year
    data["is_weekend"] = data.index.weekday
    data['is_spring'] = data['month'].isin([3, 4, 5]).astype(int)
    data['is_summer'] = data['month'].isin([6, 7, 8]).astype(int)
    data['is_fall'] = data['month'].isin([9, 10, 11]).astype(int)
    data['is_winter'] = data['month'].isin([12, 1, 2]).astype(int)
    data['sin_day'] = np.sin(2 * np.pi * data.index.dayofweek / 7)
    data['cos_day'] = np.cos(2 * np.pi * data.index.dayofweek / 7)
    return data

In [30]:
df = create_date_features(df)
df.head(5)

In [31]:
#Add holidays
 # Determine public holidays
holiday = holidays.UK()
df['is_public_holiday'] = df.index.map(lambda x: 1 if x in holiday else 0)

In [32]:
def create_lag_and_window_features(data, target_col):
    """
    Create lag and window features for a given DataFrame and target column.
    
    Parameters:
    data (pd.DataFrame): The input DataFrame.
    target_col (str): The name of the target column to create features for.

    Returns:
    pd.DataFrame: DataFrame with lag and window features.
    """
    # Create lag features for the past week
    for i in range(1, 8):
        data[f'lag_{i}'] = data[target_col].shift(i)

    # Rolling window statistics for 7 days
    data['rolling_mean_7'] = data[target_col].rolling(window=7).mean()
    data['rolling_sum_7'] = data[target_col].rolling(window=7).sum()
    data['rolling_std_7'] = data[target_col].rolling(window=7).std()

    # Rolling window statistics for 30 days
    data['rolling_mean_30'] = data[target_col].rolling(window=30).mean()
    data['rolling_sum_30'] = data[target_col].rolling(window=30).sum()
    data['rolling_std_30'] = data[target_col].rolling(window=30).std()

    # Expanding window statistics
    data['expanding_sum'] = data[target_col].expanding().sum()

    return data

In [33]:
df = create_lag_and_window_features(df, 'quantity')
df.head(5)

In [34]:
# Fill any remaining NaN values
df = df.fillna(method='bfill')
df.head(5)

## Outlier detection

In [35]:
# Line plot
plt.figure(figsize=(12, 6))
plt.plot(df['quantity'])
plt.title('Time Series Plot')
plt.show()

# Box plot
plt.figure(figsize=(12, 6))
plt.boxplot(df['quantity'], vert=False)
plt.title('Box Plot of Quantity')
plt.show();

In [36]:
# Set winsorization threshold
threshold = 60000

# Winsorize the 'quantity' column
df['quantity_winsorized'] = df['quantity'].where(df['quantity'] <= threshold, threshold)
df.head(5)

In [37]:
plt.figure(figsize=(10,6))
plt.plot(df["quantity_winsorized"]);

In [38]:
df_2010 = df[df['year'] == 2010]

# Plot the 'quantity' column for the year 2010
plt.figure(figsize=(10, 6))
plt.plot(df_2010['quantity'], label='Quantity')
plt.title('Sales Volume in 2010')
plt.xlabel('Date')
plt.ylabel('Quantity')
plt.legend()
plt.show();

In [39]:
df_2011 = df[df['year'] == 2011]

# Plot the 'quantity' column for the year 2010
plt.figure(figsize=(10, 6))
plt.plot(df_2011['quantity_winsorized'], label='Quantity')
plt.title('Sales Volume in 2011')
plt.xlabel('Date')
plt.ylabel('Quantity')
plt.legend()
plt.show();

In [42]:
df.info()

In [43]:
df.describe()

In [ ]:
df.to_csv("processed_full.csv")